In [ ]:
from entsoe import load_data, get_dap_dataloader
import numpy as np
from torch_geometric_temporal.nn.attention.stgcn import STConv
import torch
import torch.nn as nn
from tqdm import tqdm

In [ ]:
data = load_data()

In [ ]:
dataloader = get_dap_dataloader(window_size=24, future_steps=4)

In [ ]:
class FullyConnLayer(nn.Module):
    def __init__(self, c):
        super(FullyConnLayer, self).__init__()
        self.conv = nn.Conv2d(c, 1, 1)

    def forward(self, x):
        return self.conv(x)


# adapted from Hao Wei
class OutputLayer(nn.Module):
    def __init__(self, c, T, n):
        super(OutputLayer, self).__init__()
        self.tconv1 = nn.Conv2d(c, c, (T, 1), 1, dilation=1, padding=(0, 0))
        self.ln = nn.LayerNorm([n, c])
        self.tconv2 = nn.Conv2d(c, c, (1, 1), 1, dilation=1, padding=(0, 0))
        self.fc = FullyConnLayer(c)

    def forward(self, x):
        x_t1 = self.tconv1(x)
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
        x_t2 = self.tconv2(x_ln)
        return self.fc(x_t2)


class Model(nn.Module):
    def __init__(self, window_size, num_nodes):
        super(Model, self).__init__()
        self.stconv1 = STConv(
            num_nodes=10,
            in_channels=24,
            hidden_channels=16,
            out_channels=12,
            kernel_size=3,
            K=1,
        )
        self.stconv2 = STConv(
            num_nodes=10,
            in_channels=12,
            hidden_channels=16,
            out_channels=8,
            kernel_size=3,
            K=1,
        )
        # window_size - 2 * num_layers * (kernel_size - 1) = 24 - 2 * 2 * (3 - 1) = 16
        T = 24 - 2 * 2 * (3 - 1)
        self.output_layer = OutputLayer(8, T, 10)

    def forward(self, x, edge_index, edge_weight):
        x = self.stconv1(x, edge_index, edge_weight)
        x = self.stconv2(x, edge_index, edge_weight)
        x = x.permute(0, 3, 1, 2)
        x = self.output_layer(x)
        x = x.permute(0, 2, 3, 1)
        return x

In [ ]:
# len_data = len(data)
len_data = 730
window_size = 24
batch_size = 10
model = Model(window_size, 10)
model.train()
edge_index = data[0].edge_index  # static graph
# print(edge_index.shape)
criteron = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(100):
    epoch_loss = 0
    for i in tqdm(range(0, len_data - window_size - 1)):
        optimizer.zero_grad()
        window_data = data[i : i + window_size]
        X = [d.x for d in window_data]
        X = torch.stack(X, dim=0)
        # Add a 1 dimension in front as batch dimension
        X = X.unsqueeze(0)
        # print(X.shape)
        edge_weights = [d.edge_attr for d in window_data]
        avg_edge_weight = np.mean(edge_weights, axis=0)
        edge_weights = torch.tensor(avg_edge_weight, dtype=torch.float32)

        y = data[i + window_size].x[:, 0]  # first feature of a node is DAP
        # print(y.shape)
        y_hat = model(X, edge_index, edge_weights)
        # print(y_hat)
        # print(y_hat.shape)
        y_hat = y_hat.squeeze(0, 1, 3)
        # print(y_hat.shape)
        # print(y)
        loss = criteron(y_hat, y)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(epoch_loss / (len_data - window_size - 1))

In [ ]:
# len_data = len(data)
len_data = 730
window_size = 24
batch_size = 10
model = Model(window_size, 10)
model.train()
for data in dataloader:
    edge_index = data[2][0, 0, :, :]
    break
print(edge_index.shape)
criteron = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for data in tqdm(dataloader):
    optimizer.zero_grad()
    X, edge_weights, _, y = data
    print(X.shape)
    print(edge_weights.shape)
    y_hat = model(X, edge_index, edge_weights).squeeze(1, 3)
    # print(y_hat.shape)
    # print(y.shape)
    loss = criteron(y_hat, y)
    loss.backward()
    optimizer.step()
    print(loss.item())
    break